## Prepare data for model input
- Truncate all features into respective windows
- Concatenate all features in each windows
- Input data saved as .npy file

In [128]:
# Import library
import pandas as pd
import numpy as np
from sklearn import preprocessing
from scipy import stats
from tqdm import tqdm
pd.set_option('display.max_columns', None,'display.max_row', None)

In [117]:
# Import all feature files
df_funding = pd.read_csv("./cleaned_csv/cleaned_funding.csv")
df_news = pd.read_csv("./cleaned_csv/cleaned_news.csv",index_col=0)
df_static = pd.read_csv("./cleaned_csv/cleaned_static.csv")
df_investor = pd.read_csv("./cleaned_csv/cleaned_investor.csv")

In [56]:
# Set each window size as 6 months
window_size = 6

In [30]:
# Helper function 
# Generate windows
num_windows = 64
def generateWindows(df):
    all_columns = ["company_id"]
    for i in range(num_windows):
        all_columns += ['window' + str(i)]
        
    df_windows = pd.DataFrame(columns=all_columns)
    df_windows["company_id"] = df.drop_duplicates("company_id")["company_id"]
    
    return df_windows

In [ ]:
# Helper function
# Merge and truncate features into windows
def mergeFeatures(x, num_features, col_name, df_infer):
    row = []
    row = list(row)
#     print(x['company_id'])

    df_row = df_infer.loc[df_infer["company_id"] == x["company_id"]]
    max_time = df_row[col_name].max()
    
    # truncate features into respective window
    for i in range(int(max_time/window_size)+1):
        item = df_row.loc[(df_row[col_name] >= i * window_size) & 
                          (df_row[col_name] < (i+1) * window_size)]
        row.append(item.iloc[:,2:2+num_features].sum().values)
        
    # fill missing values with zeros
    for i in range(int(max_time/window_size)+1, 64):
        row.append(np.zeros(num_features))
        
    return list(row)

### Truncate funding event into windows 

In [99]:
# Normalize funding amount based on each funding round
def normalize(x):
    x = x.values.reshape(-1, 1)
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    return x_scaled

groups = df_funding.groupby("funding_round_code")["funding_amount_mil"]
for name, group in groups:
    df_funding.at[group.index,"funding_amount_mil"] = normalize(group)

/Users/zhiyun/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:355: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/Users/zhiyun/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:356: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [102]:
df_funding.drop(columns=['funding_round_code'],inplace=True)
df_funding.head()

,company_id,investment_age,funding_amount_mil,IPO,a,acquired,angel,b,c,d,e,f,merged,neeq,post-IPO,delisted,seed,stg_invse,num_investor
0,11060,139.0,0.050736,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1.0
1,11060,133.0,0.022113,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,5.0
2,11060,80.0,0.005527,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2.0
3,11060,57.0,0.000768,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4.0
4,11058,72.0,0.002552,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN


In [91]:
# Discard funding event out of 63 years timeline
df_funding = df_funding[df_funding['investment_age'] <= 383]
df_funding = df_funding[df_funding['investment_age'] >= 0]

In [64]:
df_funding['company_id'] = 'c:' + df_funding['company_id'].astype(str)
df_funding_windows = generateWindows(df_funding)
df_funding_windows.head()

,company_id,window0,window1,window2,window3,window4,window5,window6,window7,window8,window9,window10,window11,window12,window13,window14,window15,window16,window17,window18,window19,window20,window21,window22,window23,window24,window25,window26,window27,window28,window29,window30,window31,window32,window33,window34,window35,window36,window37,window38,window39,window40,window41,window42,window43,window44,window45,window46,window47,window48,window49,window50,window51,window52,window53,window54,window55,window56,window57,window58,window59,window60,window61,window62,window63
0,c:35654005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c:58166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c:64651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,c:58167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,c:47280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 17 funding features
df_funding_windows.iloc[:,1:65] = df_funding_windows.apply(lambda x: mergeFeatures(x,17,'investment_age'), 
                                                           axis=1,result_type='expand').values

In [ ]:
df_funding_window.to_pickle('./input_funding.pkl')

### Truncate news into windows 

In [84]:
df_news['company_id'] = 'c:' + df_news['company_id'].astype(str)
df_news = df_news[df_news['age_at_news'] <= 383]
df_news = df_news[df_news['age_at_news'] >= 0]
df_news.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53165 entries, 0 to 53895
Data columns (total 4 columns):
company_id       53165 non-null object
age_at_news      53165 non-null int64
pos_sentiment    53165 non-null float64
neg_sentiment    53165 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 2.0+ MB


In [85]:
df_news_windows = generateWindows(df_news)
df_news_windows.head()

,company_id,window0,window1,window2,window3,window4,window5,window6,window7,window8,window9,window10,window11,window12,window13,window14,window15,window16,window17,window18,window19,window20,window21,window22,window23,window24,window25,window26,window27,window28,window29,window30,window31,window32,window33,window34,window35,window36,window37,window38,window39,window40,window41,window42,window43,window44,window45,window46,window47,window48,window49,window50,window51,window52,window53,window54,window55,window56,window57,window58,window59,window60,window61,window62,window63
0,c:35654005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c:58166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c:64651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,c:58167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,c:47280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
# 2 news features
df_news_windows.iloc[:,1:65] = df_news_windows.apply(lambda x: mergeFeatures(x,2,'age_at_news',df_news), 
                                                           axis=1,result_type='expand').values

In [88]:
df_news_windows.to_pickle('./input_news.pkl')

### Truncate static features into windows 

In [114]:
df_static.head()

,company_id,age,from_famous_company,from_famous_school,sev_entre_exp,corporate_service,sports,agriculture,healthcare,software,marketing,real_estate,edu,media,new_industry,tourism,local_life,transportation,gaming,logistics,e-commerce,hardware,social_media,finance,first_tier
0,c:11060,155,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1,c:11058,139,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,c:11051,71,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,c:11047,182,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
4,c:11022,118,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1


In [115]:
df_static['age'][df_static['age'] > 383] = 383  # modify company age exceeds 64*6 to fit windows
df_static_windows = generateWindows(df_static)
df_static_windows.head()

/Users/zhiyun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,company_id,window0,window1,window2,window3,window4,window5,window6,window7,window8,window9,window10,window11,window12,window13,window14,window15,window16,window17,window18,window19,window20,window21,window22,window23,window24,window25,window26,window27,window28,window29,window30,window31,window32,window33,window34,window35,window36,window37,window38,window39,window40,window41,window42,window43,window44,window45,window46,window47,window48,window49,window50,window51,window52,window53,window54,window55,window56,window57,window58,window59,window60,window61,window62,window63
0,c:11060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c:11058,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c:11051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,c:11047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,c:11022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 23 static features
df_news_windows.iloc[:,1:65] = df_news_windows.apply(lambda x: mergeFeatures(x,23,'age_at_news',df_news), 
                                                           axis=1,result_type='expand').values

In [ ]:
df_static_window.to_pickle('./input_static.pkl')

### Truncate investor features into windows 

In [129]:
df_investor['company_id'] = 'c:' + df_investor['company_id'].astype(str)

In [132]:
df_investor = df_investor[df_investor['investment_age'] <= 383]
df_investor = df_investor[df_investor['investment_age'] >= 0]

In [131]:
df_investor_windows = generateWindows(df_investor)
df_investor_windows.head()

,company_id,window0,window1,window2,window3,window4,window5,window6,window7,window8,window9,window10,window11,window12,window13,window14,window15,window16,window17,window18,window19,window20,window21,window22,window23,window24,window25,window26,window27,window28,window29,window30,window31,window32,window33,window34,window35,window36,window37,window38,window39,window40,window41,window42,window43,window44,window45,window46,window47,window48,window49,window50,window51,window52,window53,window54,window55,window56,window57,window58,window59,window60,window61,window62,window63
0,c:11060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,c:11058,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,c:11051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,c:11047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,c:11022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 28 investor features
df_investor_windows.iloc[:,1:65] = df_investor_windows.apply(lambda x: mergeFeatures(x,28,'investment_age',df_investor), 
                                                             axis=1,result_type='expand').values

In [ ]:
df_static_window.to_pickle('./input_investor.pkl')

### Merge all pickle files

In [3]:
df_funding = pd.read_pickle('./input_funding.pkl')
df_news = pd.read_pickle('./input_news.pkl')
df_static = pd.read_pickle('./input_static.pkl')   #'./new_input_static_with_gps_location.pkl'
df_investor = pd.read_pickle('./input_investor.pkl')

print('Funding features:',df_funding.iloc[1,1].shape[0])
print('News features:',df_news.iloc[1,1].shape[0])
print('Statis features:',df_static.iloc[1,1].shape[0])
print('Investor features:',df_investor.iloc[1,1].shape[0])

Funding features: 17
News features: 2
Statis features: 23
Investor features: 28


In [15]:
print('Null value for funding data:',df_funding.isnull().values.any())
print('Null value for news data:',df_news.isnull().values.any())
print('Null value for static data:',df_static.isnull().values.any())
print('Null value for investor data:',df_investor.isnull().values.any())
print()
print('Shape of funding data:',df_funding.shape)
print('Shape of news data:',df_news.shape)
print('Shape of static data:',df_static.shape)
print('Shape of investor data:',df_investor.shape)

Null value for funding data: False
Null value for news data: False
Null value for static data: False
Null value for investor data: False

Shape of funding data: (38812, 65)
Shape of news data: (25844, 65)
Shape of static data: (38812, 65)
Shape of investor data: (38812, 65)


In [384]:
# merge all pickle files
dfs = [df_funding, df_static, df_news, df_investor]
df_merged = reduce(lambda left,right: pd.merge(left,right,on=['company_id'],how='left'), dfs)
df_merged = df_merged[df_merged.columns.drop(list(df_merged.filter(regex='63')))]

print('Total number of samples:', df_merged.shape[0])
print('Total number of windows:', df_merged.shape[1])

Total number of samples: 38812
Total number of windows: 257


In [387]:
# fill zeros for company without news
def fillzeros(x, num):
    if np.isnan(x).all():
        x = np.zeros(num)
    return x

for i in range(129,193):
    df_merged.iloc[:,i] = df_merged.iloc[:,i].apply(fillzeros,num=2)

In [390]:
df_merged

,company_id,window0_x,window1_x,window2_x,window3_x,window4_x,window5_x,window6_x,window7_x,window8_x,window9_x,window10_x,window11_x,window12_x,window13_x,window14_x,window15_x,window16_x,window17_x,window18_x,window19_x,window20_x,window21_x,window22_x,window23_x,window24_x,window25_x,window26_x,window27_x,window28_x,window29_x,window30_x,window31_x,...,window30_y,window31_y,window32_y,window33_y,window34_y,window35_y,window36_y,window37_y,window38_y,window39_y,window40_y,window41_y,window42_y,window43_y,window44_y,window45_y,window46_y,window47_y,window48_y,window49_y,window50_y,window51_y,window52_y,window53_y,window54_y,window55_y,window56_y,window57_y,window58_y,window59_y,window60_y,window61_y,window62_y
0,c:11060,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0007684621295862535, 0.0, 1.0, 0.0, 0.0, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.005526534070204542, 0.0, 0.0, 0.0, 0.0, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.022113202948427058, 0.0, 0.0, 0.0, 0.0, 0.0...","[0.05073578287800347, 0.0, 0.0, 1.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [392]:
df_merged.shape

(38812, 253)

In [1]:
# concatenate all features into respective windows
num_samples = 38812
timestep = 63
num_features = 17+2+23+28

df_input = np.zeros((num_samples, timestep, num_features)) 
df_merged_values = df_merged.values

for i in range(timestep):
    for j in range(num_samples):
            df_input[j, i] = np.concatenate([df_merged_values[j, i+1], 
                                             df_merged_values[j, i+1+63],
                                             df_merged_values[j, i+1+63*2],
                                             df_merged_values[j, i+1+63*3]])

print('Input data shape:', df_input.shape)

Input data shape: (38812, 63, 70)


In [396]:
np.save('X.npy', df_input)